# Lab 4: Physics-Informed Neural Networks (PINNs)
## Heat Equation with PyTorch

**Name:** _________________________

**Date:** _________________________

**Lab Partner:** _________________________

---

## Learning Objectives
By completing this lab, you will:
- Understand what makes PINNs different from traditional methods
- Implement a PINN from scratch using PyTorch
- Use automatic differentiation to compute PDE residuals
- Train a neural network to solve a time-dependent PDE
- Compare PINN solutions to analytical solutions
- Gain practical experience for Project 1

---
## Setup: Import Libraries

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Print versions
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print("✓ All packages loaded successfully!")

---
# Part 1: Understanding the Heat Equation

## Problem Setup

We'll solve the 1D heat equation:

$$\frac{\partial u}{\partial t} = 0.1 \frac{\partial^2 u}{\partial x^2}, \quad x \in [0, 1], \, t \in [0, 0.5]$$

**Initial condition:** $u(x, 0) = \sin(\pi x)$

**Boundary conditions:** $u(0, t) = 0, \quad u(1, t) = 0$

**Analytical solution:** $u(x, t) = e^{-0.1 \pi^2 t} \sin(\pi x)$

## Exercise 1.1: Visualize the Analytical Solution

In [ ]:
# Parameters
alpha = 0.1      # Thermal diffusivity
L = 1.0          # Rod length
T_max = 0.5      # Maximum time

# Create spatial grid
x = np.linspace(0, L, 100)

# Function to compute analytical solution
def analytical_solution(x, t, alpha):
    """
    Analytical solution to heat equation
    
    Args:
        x: Position (array or scalar)
        t: Time (scalar)
        alpha: Thermal diffusivity
    
    Returns:
        Temperature u(x,t)
    """
    return np.exp(-alpha * np.pi**2 * t) * np.sin(np.pi * x)

# Plot at different times
times = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
colors = plt.cm.hot(np.linspace(0.3, 1, len(times)))

plt.figure(figsize=(10, 6))

for t, color in zip(times, colors):
    u = analytical_solution(x, t, alpha)
    plt.plot(x, u, label=f't = {t:.1f}s', color=color, linewidth=2)

plt.xlabel('Position x (m)', fontsize=12)
plt.ylabel('Temperature u(x,t) (°C)', fontsize=12)
plt.title('Analytical Solution: Heat Diffusion in a Rod', fontsize=14, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('analytical_solution.png', dpi=150)
plt.show()

## Exercise 1.2: Understanding the Physics

**Discuss with your lab partner and answer:**

1. **How does the maximum temperature change over time?**
   - Answer: _______________________________________________

2. **Why do the ends stay at zero?**
   - Answer: _______________________________________________

3. **What happens as t → ∞?**
   - Answer: _______________________________________________

4. **What would change if we increased α?**
   - Answer: _______________________________________________

---
# Part 2: Building a PINN from Scratch

## The PINN Recipe (3 Steps)

1. **Represent solution as neural network:** $u(x,t) \approx u_\theta(x,t)$
2. **Define loss function:** $\mathcal{L} = \|\text{PDE residual}\|^2 + \|\text{BC violation}\|^2 + \|\text{IC violation}\|^2$
3. **Train network to minimize loss:** $\theta_{\text{new}} = \theta_{\text{old}} - \eta \nabla_\theta \mathcal{L}$

## Step 1: Define the Neural Network

In [ ]:
class HeatPINN(nn.Module):
    """
    Neural network to approximate u(x, t)
    
    Architecture:
        Input:  [x, t] (2D)
        Hidden: 4 layers × 20 neurons each
        Output: u (1D)
    """
    def __init__(self, hidden_layers=4, neurons_per_layer=20):
        super(HeatPINN, self).__init__()
        
        # Build layers list
        layers = []
        
        # Input layer: 2 inputs (x, t) → neurons_per_layer
        layers.append(nn.Linear(2, neurons_per_layer))
        layers.append(nn.Tanh())
        
        # Hidden layers
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(nn.Tanh())
        
        # Output layer: neurons_per_layer → 1 output (u)
        layers.append(nn.Linear(neurons_per_layer, 1))
        
        # Combine into sequential network
        self.network = nn.Sequential(*layers)
    
    def forward(self, x, t):
        """
        Forward pass through network
        
        Args:
            x: Spatial coordinate (N × 1 tensor)
            t: Time coordinate (N × 1 tensor)
        
        Returns:
            u: Temperature predictions (N × 1 tensor)
        """
        # Concatenate x and t into single input
        inputs = torch.cat([x, t], dim=1)  # Shape: (N, 2)
        return self.network(inputs)

# Create model
model = HeatPINN(hidden_layers=4, neurons_per_layer=20)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"✓ Model created with {num_params} trainable parameters")

# Test forward pass
x_test = torch.tensor([[0.5]])  # Middle of rod
t_test = torch.tensor([[0.0]])  # At t=0
u_test = model(x_test, t_test)
print(f"✓ Forward pass works! u(0.5, 0) = {u_test.item():.4f} (random initialization)")

## Exercise 2.1: Understanding the Architecture

1. **How many inputs does the network have? Why?**
   - Answer: _______________________________________________

2. **How many outputs? Why?**
   - Answer: _______________________________________________

3. **Why do we use Tanh activation instead of ReLU?**
   - Answer: _______________________________________________

4. **What would happen if we used only 1 hidden layer?**
   - Answer: _______________________________________________

## Step 2: Compute PDE Residual (The Magic!)

In [ ]:
def compute_pde_residual(model, x, t, alpha):
    """
    Compute PDE residual using automatic differentiation
    
    Args:
        model: Neural network
        x: Spatial points (N × 1 tensor, requires_grad=True)
        t: Time points (N × 1 tensor, requires_grad=True)
        alpha: Thermal diffusivity
    
    Returns:
        residual: du/dt - alpha * d²u/dx² (N × 1 tensor)
    """
    # Ensure gradients are enabled
    x.requires_grad_(True)
    t.requires_grad_(True)
    
    # Forward pass: compute u(x,t)
    u = model(x, t)
    
    # First derivatives using automatic differentiation
    u_t = torch.autograd.grad(
        outputs=u,
        inputs=t,
        grad_outputs=torch.ones_like(u),
        create_graph=True,      # Keep graph for higher derivatives
        retain_graph=True       # Don't destroy graph
    )[0]
    
    u_x = torch.autograd.grad(
        outputs=u,
        inputs=x,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # Second derivative: d²u/dx²
    u_xx = torch.autograd.grad(
        outputs=u_x,
        inputs=x,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # Compute residual: du/dt - alpha * d²u/dx²
    residual = u_t - alpha * u_xx
    
    return residual

# Test it!
x_test = torch.tensor([[0.5]], requires_grad=True)
t_test = torch.tensor([[0.1]], requires_grad=True)

residual = compute_pde_residual(model, x_test, t_test, alpha=0.1)
print(f"✓ PDE residual at (x=0.5, t=0.1): {residual.item():.6f}")
print("  (Should be close to 0 after training!)")

## Exercise 2.2: Understanding Automatic Differentiation

1. **Why do we need `create_graph=True`?**
   - Answer: _______________________________________________

2. **What happens if we forget it when computing u_xx?**
   - Answer: _______________________________________________

3. **What does `grad_outputs=torch.ones_like(u)` do?**
   - Answer: _______________________________________________

## Step 3: Create Training Data (Collocation Points)

In [ ]:
def create_training_data(n_domain=1000, n_boundary=100, n_initial=100):
    """
    Create collocation points for training
    
    Returns:
        Dictionary with 'domain', 'boundary', 'initial' point sets
    """
    # Domain points: random in [0,1] × [0,0.5]
    x_domain = torch.rand(n_domain, 1)
    t_domain = torch.rand(n_domain, 1) * 0.5
    
    # Boundary points: x=0 and x=1 for various t
    t_boundary = torch.rand(n_boundary, 1) * 0.5
    
    # Left boundary (x=0)
    x_boundary_left = torch.zeros(n_boundary // 2, 1)
    t_boundary_left = t_boundary[:n_boundary // 2]
    
    # Right boundary (x=1)
    x_boundary_right = torch.ones(n_boundary // 2, 1)
    t_boundary_right = t_boundary[n_boundary // 2:]
    
    # Combine boundaries
    x_boundary = torch.cat([x_boundary_left, x_boundary_right])
    t_boundary = torch.cat([t_boundary_left, t_boundary_right])
    
    # Initial condition points: t=0 for various x
    x_initial = torch.rand(n_initial, 1)
    t_initial = torch.zeros(n_initial, 1)
    
    return {
        'domain': (x_domain, t_domain),
        'boundary': (x_boundary, t_boundary),
        'initial': (x_initial, t_initial)
    }

# Create data
data = create_training_data(n_domain=1000, n_boundary=100, n_initial=100)

print(f"✓ Created training data:")
print(f"  Domain points:   {data['domain'][0].shape[0]}")
print(f"  Boundary points: {data['boundary'][0].shape[0]}")
print(f"  Initial points:  {data['initial'][0].shape[0]}")

In [ ]:
# Visualize collocation points
plt.figure(figsize=(10, 6))

plt.scatter(data['domain'][0].numpy(), data['domain'][1].numpy(), 
           s=2, alpha=0.3, c='blue', label='Domain (PDE enforced)')
plt.scatter(data['boundary'][0].numpy(), data['boundary'][1].numpy(), 
           s=20, c='red', marker='s', label='Boundary (BC enforced)')
plt.scatter(data['initial'][0].numpy(), data['initial'][1].numpy(), 
           s=20, c='green', marker='^', label='Initial (IC enforced)')

plt.xlabel('Position x (m)', fontsize=12)
plt.ylabel('Time t (s)', fontsize=12)
plt.title('Collocation Points for PINN Training', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 0.55)
plt.tight_layout()
plt.savefig('collocation_points.png', dpi=150)
plt.show()

## Step 4: Define Loss Function

In [ ]:
def compute_loss(model, data, alpha):
    """
    Compute total loss = PDE loss + BC loss + IC loss
    
    Args:
        model: Neural network
        data: Dictionary with training data
        alpha: Thermal diffusivity
    
    Returns:
        Tuple of (total_loss, pde_loss, bc_loss, ic_loss)
    """
    x_domain, t_domain = data['domain']
    x_boundary, t_boundary = data['boundary']
    x_initial, t_initial = data['initial']
    
    # 1. PDE Loss: Enforce heat equation in interior
    residual = compute_pde_residual(model, x_domain, t_domain, alpha)
    loss_pde = torch.mean(residual**2)
    
    # 2. Boundary Condition Loss: u(0,t) = u(1,t) = 0
    u_boundary = model(x_boundary, t_boundary)
    loss_bc = torch.mean(u_boundary**2)
    
    # 3. Initial Condition Loss: u(x,0) = sin(πx)
    u_initial = model(x_initial, t_initial)
    u_initial_true = torch.sin(np.pi * x_initial)
    loss_ic = torch.mean((u_initial - u_initial_true)**2)
    
    # Total loss (could add weights here)
    loss_total = loss_pde + loss_bc + loss_ic
    
    return loss_total, loss_pde, loss_bc, loss_ic

# Test loss computation
loss_total, loss_pde, loss_bc, loss_ic = compute_loss(model, data, alpha=0.1)

print(f"✓ Loss computation works!")
print(f"  Initial losses (before training):")
print(f"    Total:    {loss_total.item():.6f}")
print(f"    PDE:      {loss_pde.item():.6f}")
print(f"    BC:       {loss_bc.item():.6f}")
print(f"    IC:       {loss_ic.item():.6f}")
print(f"\n  Goal: All losses → 0 after training!")

## Exercise 2.3: Loss Function Analysis

1. **Why do we square the residuals before taking the mean?**
   - Answer: _______________________________________________

2. **What would happen if we removed the IC loss?**
   - Answer: _______________________________________________

3. **Should all three loss terms have equal weight? Why or why not?**
   - Answer: _______________________________________________

## Step 5: Training Loop

In [ ]:
def train_pinn(model, data, alpha, epochs=5000, lr=0.001):
    """
    Train the PINN by minimizing loss
    
    Args:
        model: Neural network
        data: Training data dictionary
        alpha: Thermal diffusivity
        epochs: Number of training iterations
        lr: Learning rate
    
    Returns:
        Dictionary with loss history
    """
    # Set up optimizer (Adam = fancy gradient descent)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Storage for loss history
    history = {'total': [], 'pde': [], 'bc': [], 'ic': []}
    
    print("Starting training...")
    print(f"{'Epoch':>7} | {'Total Loss':>12} | {'PDE Loss':>12} | {'BC Loss':>12} | {'IC Loss':>12}")
    print("-" * 70)
    
    for epoch in range(epochs):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        
        # Compute loss
        loss_total, loss_pde, loss_bc, loss_ic = compute_loss(model, data, alpha)
        
        # Backpropagation: compute gradients
        loss_total.backward()
        
        # Update parameters: θ_new = θ_old - lr * ∇L
        optimizer.step()
        
        # Record history
        history['total'].append(loss_total.item())
        history['pde'].append(loss_pde.item())
        history['bc'].append(loss_bc.item())
        history['ic'].append(loss_ic.item())
        
        # Print progress every 500 epochs
        if epoch % 500 == 0 or epoch == epochs - 1:
            print(f"{epoch:7d} | {loss_total.item():12.6e} | "
                  f"{loss_pde.item():12.6e} | "
                  f"{loss_bc.item():12.6e} | "
                  f"{loss_ic.item():12.6e}")
    
    print("\n✓ Training complete!")
    return history

# Train the model!
history = train_pinn(model, data, alpha=0.1, epochs=5000, lr=0.001)

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total loss (log scale)
axes[0].semilogy(history['total'], 'b-', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Total Loss (log scale)', fontsize=12)
axes[0].set_title('Total Loss During Training', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Individual components
axes[1].semilogy(history['pde'], label='PDE Loss', linewidth=2)
axes[1].semilogy(history['bc'], label='BC Loss', linewidth=2)
axes[1].semilogy(history['ic'], label='IC Loss', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss (log scale)', fontsize=12)
axes[1].set_title('Loss Components', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

## Exercise 2.4: Training Dynamics

1. **Which loss component decreases fastest? Why?**
   - Answer: _______________________________________________

2. **Does the total loss reach exactly zero? Should it?**
   - Answer: _______________________________________________

3. **What would you try if training didn't converge?**
   - Answer: _______________________________________________

---
# Part 3: Evaluation and Comparison

## Step 1: Generate Predictions on a Grid

In [ ]:
def evaluate_pinn(model, nx=100, nt=100):
    """
    Evaluate PINN on a regular grid for visualization
    
    Args:
        model: Trained neural network
        nx: Number of spatial points
        nt: Number of time points
    
    Returns:
        X, T, u_pred: Meshgrids and predictions
    """
    # Create test grid
    x = torch.linspace(0, 1, nx).reshape(-1, 1)
    t = torch.linspace(0, 0.5, nt).reshape(-1, 1)
    
    # Create meshgrid
    X, T = torch.meshgrid(x.squeeze(), t.squeeze(), indexing='ij')
    x_flat = X.reshape(-1, 1)
    t_flat = T.reshape(-1, 1)
    
    # Predictions (no gradients needed)
    with torch.no_grad():
        u_pred = model(x_flat, t_flat).reshape(X.shape)
    
    return X.numpy(), T.numpy(), u_pred.numpy()

# Evaluate PINN
print("Evaluating PINN on test grid...")
X, T, u_pred = evaluate_pinn(model, nx=100, nt=100)

# Compute analytical solution
u_true = analytical_solution(X, T, alpha=0.1)

# Compute error
error = np.abs(u_pred - u_true)

print(f"✓ Evaluation complete!")
print(f"  Grid size: {X.shape}")
print(f"  Total test points: {X.size}")

## Step 2: Visualize Results (The Moment of Truth!)

In [ ]:
# Create comprehensive comparison plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# PINN solution
im1 = axes[0].contourf(X, T, u_pred, levels=50, cmap='hot')
axes[0].set_xlabel('Position x (m)', fontsize=11)
axes[0].set_ylabel('Time t (s)', fontsize=11)
axes[0].set_title('PINN Solution', fontsize=13, fontweight='bold')
cbar1 = plt.colorbar(im1, ax=axes[0])
cbar1.set_label('Temperature (°C)', fontsize=10)

# Analytical solution
im2 = axes[1].contourf(X, T, u_true, levels=50, cmap='hot')
axes[1].set_xlabel('Position x (m)', fontsize=11)
axes[1].set_ylabel('Time t (s)', fontsize=11)
axes[1].set_title('Analytical Solution', fontsize=13, fontweight='bold')
cbar2 = plt.colorbar(im2, ax=axes[1])
cbar2.set_label('Temperature (°C)', fontsize=10)

# Absolute error
im3 = axes[2].contourf(X, T, error, levels=50, cmap='viridis')
axes[2].set_xlabel('Position x (m)', fontsize=11)
axes[2].set_ylabel('Time t (s)', fontsize=11)
axes[2].set_title('Absolute Error', fontsize=13, fontweight='bold')
cbar3 = plt.colorbar(im3, ax=axes[2])
cbar3.set_label('|u_PINN - u_true| (°C)', fontsize=10)

plt.tight_layout()
plt.savefig('solution_comparison.png', dpi=150)
plt.show()

# Print error statistics
print(f"\n{'='*50}")
print(f"ERROR STATISTICS")
print(f"{'='*50}")
print(f"  Max error:        {np.max(error):.6f}")
print(f"  Mean error:       {np.mean(error):.6f}")
print(f"  L2 error:         {np.sqrt(np.mean(error**2)):.6f}")
print(f"  Relative L2 (%):  {100 * np.linalg.norm(error) / np.linalg.norm(u_true):.2f}%")
print(f"{'='*50}")

## Step 3: Temporal Snapshots

In [ ]:
# Plot solution profiles at different times
fig, ax = plt.subplots(figsize=(12, 7))

times = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
colors = plt.cm.hot(np.linspace(0.3, 1, len(times)))
x_plot = X[:, 0]

for i, (t_val, color) in enumerate(zip(times, colors)):
    # Find closest time index
    t_idx = np.argmin(np.abs(T[0, :] - t_val))
    
    # Plot PINN (solid line with markers)
    ax.plot(x_plot, u_pred[:, t_idx], 'o-', 
           label=f't={t_val:.1f}s (PINN)', 
           color=color, markersize=5, linewidth=2.5, alpha=0.9)
    
    # Plot analytical (dashed line)
    ax.plot(x_plot, u_true[:, t_idx], '--', 
           color=color, linewidth=2, alpha=0.6)

# Add legend
from matplotlib.lines import Line2D
custom_lines = [
    Line2D([0], [0], color='black', linewidth=2.5, linestyle='-'),
    Line2D([0], [0], color='black', linewidth=2, linestyle='--')
]
legend1 = ax.legend(custom_lines, ['PINN', 'Analytical'], 
                   loc='upper right', fontsize=11, title='Method')

ax.set_xlabel('Position x (m)', fontsize=13)
ax.set_ylabel('Temperature u(x,t) (°C)', fontsize=13)
ax.set_title('Temperature Evolution: Comparing PINN vs Analytical', 
            fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.02, 1.02)

plt.tight_layout()
plt.savefig('temporal_snapshots.png', dpi=150)
plt.show()

## Exercise 3.1: Results Analysis

1. **Where is the error largest? Why might this be?**
   - Answer: _______________________________________________

2. **How does the relative L2 error compare to typical finite difference accuracy?**
   - Answer: _______________________________________________

3. **Does the PINN respect physics at t=0.5? Does it look reasonable?**
   - Answer: _______________________________________________

4. **What do you think would happen if we trained for 50,000 epochs instead of 5,000?**
   - Answer: _______________________________________________

---
# Part 4: Experimentation

Try modifying the PINN to see what happens! **This is the fun part!**

## Exercise 4.1: Architecture Exploration

Change the network architecture and retrain. Try at least 2 configurations:

In [ ]:
# Configuration 1: Fewer layers
# TODO: Implement and train model_small
# model_small = HeatPINN(hidden_layers=2, neurons_per_layer=20)
# history_small = train_pinn(model_small, data, alpha=0.1, epochs=5000, lr=0.001)
# Evaluate and compute error...

In [ ]:
# Configuration 2: More neurons
# TODO: Implement and train model_wide
# model_wide = HeatPINN(hidden_layers=4, neurons_per_layer=50)
# history_wide = train_pinn(model_wide, data, alpha=0.1, epochs=5000, lr=0.001)
# Evaluate and compute error...

**Record your results:**

Original (4 layers × 20 neurons, Tanh):
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____
- Relative L2: _____

Configuration 1: [your description]
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____
- Relative L2: _____

Configuration 2: [your description]
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____
- Relative L2: _____

**Discussion:** Which architecture worked best? Why do you think that is?

## Exercise 4.2: Collocation Point Sensitivity

In [ ]:
# TODO: Try different numbers of collocation points
# Fewer points:
# data_sparse = create_training_data(n_domain=500, n_boundary=50, n_initial=50)
# history_sparse = train_pinn(model, data_sparse, alpha=0.1, epochs=5000, lr=0.001)

# More points:
# data_dense = create_training_data(n_domain=2000, n_boundary=200, n_initial=200)
# history_dense = train_pinn(model, data_dense, alpha=0.1, epochs=5000, lr=0.001)

**Record your results:**

Fewer points (500/50/50):
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____

Original (1000/100/100):
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____

More points (2000/200/200):
- Training time: _____ seconds
- Final total loss: _____
- Max error: _____

**Discussion:** Is there a point of diminishing returns? What's the sweet spot?

---
# Part 5: Reflection and Connection to Project 1

## Exercise 5.1: Project Planning

Based on this lab experience, answer these questions:

**1. Which PDE are you leaning toward for Project 1?**
- [ ] Wave Equation
- [ ] Burgers' Equation
- [ ] Laplace/Poisson Equation

Why this choice?

_________________________________________________________________________

_________________________________________________________________________

**2. What challenges do you anticipate in implementing your chosen PDE?**

_________________________________________________________________________

_________________________________________________________________________

**3. From today's lab, what would you do differently in your project?**

_________________________________________________________________________

_________________________________________________________________________

**4. What surprised you most about PINNs?**

_________________________________________________________________________

_________________________________________________________________________

**5. When would you choose PINN vs traditional finite difference?**

Use PINN when:

_________________________________________________________________________

Use finite difference when:

_________________________________________________________________________

---
# Submission Checklist

Before submitting, make sure you have:

- [ ] Completed all exercises with answers filled in
- [ ] All code cells run without errors
- [ ] All five figures saved:
  - [ ] `analytical_solution.png`
  - [ ] `collocation_points.png`
  - [ ] `training_history.png`
  - [ ] `solution_comparison.png`
  - [ ] `temporal_snapshots.png`
- [ ] Completed Part 5 reflection questions
- [ ] Restarted kernel and ran all cells (Kernel → Restart & Run All)

**Submit via Canvas:**
1. This notebook (renamed to `lab4_[YourName].ipynb`)
2. All five PNG figures
3. Brief reflection (1 paragraph) in a separate document

---
# Resources

**PyTorch Tutorials:**
- Autograd tutorial: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html
- Neural networks: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

**PINN Papers:**
- Original paper: Raissi et al., *J. Comput. Phys.* 378 (2019)
  - https://www.sciencedirect.com/science/article/pii/S0021999118307125

**DeepXDE:**
- Documentation: https://deepxde.readthedocs.io/
- Examples: https://github.com/lululxvi/deepxde/tree/master/examples

**Office Hours:**
- Tuesdays 2-4pm
- Thursdays 10-12pm
- Or by appointment